In [1]:
from neo4j import GraphDatabase
import json
from pprint import pprint

driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', '1234'))

In [ ]:
def load_data_query():
    query = '''
        LOAD CSV WITH HEADERS FROM 'file:///food.csv' AS r
        MERGE (item: Item { name:r.name, price: toInteger(r.price), stock: toInteger(r.stock), img: r.link })
        MERGE (brand: Brand { brand: r.brand })
        FOREACH(label IN split(r.labels, ";")|MERGE (l:Label { name: label }))

        WITH r
        MATCH (item: Item { name: r.name })
        UNWIND split(r.labels, ";") as l
        MATCH (label: Label { name: l })
        MERGE (item)-[:is]-(label)

        WITH r
        MATCH (item: Item { name: r.name })
        MATCH (brand: Brand { brand:r.brand })
        MERGE (item)-[:by]-(brand)
    '''

    with driver.session() as session:
        results = session.run(query)

    return results

In [31]:
def add_item_query(name, price, stock, image, brand, labels):
    get_item_query = '''
        MATCH(item: Item { name: $name })
        RETURN item
    '''

    add_item_query = '''
        CREATE (item: Item { name: $name })
        SET item.price = $price, item.stock = $stock, item.img = $image
        MERGE (brand: Brand { brand: $brand })
        FOREACH(label IN $labels | MERGE (l:Label { name: label }))

        WITH item
        UNWIND $labels as l
        MATCH (label: Label { name: l })
        MERGE (item)-[:is]-(label)

        WITH item
        MATCH (item: Item { name: $name })
        MATCH (brand: Brand { brand: $brand })
        MERGE (item)-[:by]-(brand)

        RETURN DISTINCT item
    '''

    with driver.session() as session:
        tx = session.begin_transaction()
        results = tx.run(get_item_query, name=name)

        if list(results): 
            tx.rollback()

        else: 
            results = tx.run(add_item_query, name=name, price=price, stock=stock, image=image, brand=brand, labels=labels)
            tx.commit()

    return results.single().value()

In [ ]:
def update_labels_query(id, labels):
    query = '''
        MATCH (item:Item)
        WHERE ID(item) = $id
        FOREACH(label IN $labels | MERGE (l:Label { name: label }))

        WITH item
        UNWIND $labels$ as l
        MATCH (label: Label { name: l })
        MERGE (item)-[:is]-(label)

        RETURN item
    '''

    with driver.session() as session:
        results = session.run(query, id=id, labels=labels)

    return results.single().value()

In [43]:
def update_item_query(id, property, value):
    query = '''
        MATCH (item:Item)
        WHERE ID(item) = $id
        CALL apoc.create.setProperty(item, $property, $value)
        YIELD node
        RETURN item
    '''

    with driver.session() as session:
        results = session.run(query, id=id, value=value, property=property)

    return results.single().value()

In [ ]:
def update_item_amount_query(id, stock):
    query = '''
        MATCH (item:Item)
        WHERE ID(item) = $id
        SET item.stock = $stock
        RETURN item
    '''

    with driver.session() as session:
        results = session.run(query, id=id, stock=stock)

    return results.single().value()

In [ ]:
def remove_unused_labels_query():
    query = '''
        MATCH (label:Label)
        WHERE NOT (label)-[:is]-(:Item)
        DELETE label
        RETURN "SUCCESS"
    '''

    with driver.session() as session:
        results = session.run(query)

    return results

In [ ]:
def filter_by_label_query(label):
    query = '''
        match (:Label { name: $label })--(item:Item)
        return item.name
    '''

    with driver.session() as session:
        results = session.run(query, label=label)

    return results

In [ ]:
def filter_by_favorits_query(favorits):
    query = '''
        UNWIND $favorits as fav
        MATCH (:Label { name: fav })--(item:Item)
        RETURN item
    '''

    with driver.session() as session:
        results = session.run(query, favorits=favorits)

    return results

In [ ]:
def order_by_price_query()):
    query = '''
        MATCH (item:Item)
        RETURN item
        ORDER BY item.price
    '''

    with driver.session() as session:
        results = session.run(query)

    return results

In [ ]:
def filter_price_query(minimum, maximum):
    query = '''
        MATCH (item:Item)
        WHERE item.price > $minimum and item.price < $maximum
        RETURN item
    '''

    with driver.session() as session:
        results = session.run(query, minimum=minimum, maximum=maximum)

    return results

In [8]:
def get_items_query(item_ids):
    query = '''
        MATCH (item: Item)
        WHERE ID(item) IN $item_ids
        RETURN item
    '''

    with driver.session() as session:
        results = session.run(query, item_ids=item_ids)

    return results

In [5]:
def place_order(order):
    query = '''
        UNWIND $items as arg_item
        MERGE (item: Item)
        WITH arg_item, item
        WHERE ID(item) = arg_item.id
        SET item.stock = item.stock - arg_item.amount
        RETURN item
    '''

    order_items = [{ 'id': key, 'amount': item } for key, item in order.items()]

    with driver.session() as session:
        tx = session.begin_transaction()
        results = session.run(query, items=order_items)
        items = [result['item'] for result in results]

        if len(items) != len(order): 
            #raise Exception('Invalid Input Exception', 'Running Transaction Rollback')
            tx.rollback()
            return None

        else:
            tx.commit()
            return items

In [6]:
order = {1: 1, 9999: 2}
results = place_order(order)

for result in results:
    print(result)

Exception: ('Invalid Input Exception', 'Running Transaction Rollback')

In [32]:
# results = add_item('ULTRA Drik', 10000, 25, 'img url', 'Jacobs Sodavand', ['Drinks', 'Discount'])
# 
# for result in results:
#     print(result['item'])

<Node id=347 labels={'Item'} properties={'name': 'ULTRA Drik', 'img': 'img url', 'stock': 25, 'price': 10000}>
